# **Save** **files** **from** **URL** **or** **Torrent** **to** **any** **Cloud** **Drive**

In [ ]:
#@title <<----<strong>Install rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

clear_output()
print('---INSTALLED RCLONE---')

In [ ]:
#@title <-----<strong>Upload your  Rclone.conf file</strong>
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")


clear_output()

print('---UPLOADED CONFIG FILE---')

Saving rclone.conf to rclone.conf
User uploaded file "rclone.conf" with length 3244 bytes
Moved rclone.conf to /root/.config/rclone/rclone.conf


# <center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

clear_output()
print('---MOUNTED DRIVE---')

Mount success! - 	/content/drives/OneDrive
2021/11/13 17:07:51 Fatal error: Directory already mounted, use --allow-non-empty to mount anyway: /content/drives/OneDrive
Mount success! - 	/content/drives/OneDrive


In [ ]:
#@title **1 - QBitTorrent INITIATION**

from google.colab import drive, output
import os

print('----- Mount Drive -----')
drive._mount('/content/drive')
print('----- Installing Qbittorrent -----')
if not os.path.isfile('/usr/bin/qbittorrent-nox'):
    !apt install qbittorrent-nox
output.clear()
print('----- Done -----')

In [ ]:
#@title **2 - START QBITTORENT**
SAVE_PATH = '/content/drive/MyDrive' #@param {type:"string"}
#@markdown > *Note: Don't terminate the colab right after you're done torrenting. It takes time to write to GDrive*

if not os.path.isdir(SAVE_PATH):
    print('Save path invalid')
else:
    !mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/zn6HBgHz -o /root/.config/qBittorrent/qBittorrent.conf
    with open('/root/.config/qBittorrent/qBittorrent.conf', 'r') as f:
        content = f.read().replace('%SavePath%', SAVE_PATH)
    with open('/root/.config/qBittorrent/qBittorrent.conf', 'w') as f:
        f.write(content)
    !qbittorrent-nox -d --webui-port=4444
    print('----- Open this link to open QBittorrent -----')
    print(output.eval_js("google.colab.kernel.proxyPort(4444)"))

In [ ]:
#@title **3 - INFINITY RUN**
while True:pass

In [ ]:
#@title <-----<strong> Direct Download to Drive</strong>

import os
from os import makedirs
makedirs("/content/Downloaded", exist_ok = True)
os.chdir("/content/Downloaded")

location = "/content/drives/OneDrive/Downloads" #@param {type:"string"}

link = "" #@param {type:"string"}

wgetORcurl_command = f'wget -c + " {link} " --no-check-certificate'

!$wgetORcurl_command

!rclone -v move "/content/Downloaded/" "$location"

--2021-11-13 17:08:46--  https://www.appsloveworld.com/wp-content/uploads/2018/10/640.mp4
Resolving www.appsloveworld.com (www.appsloveworld.com)... 52.12.198.95, 100.21.184.71, 35.155.25.163
Connecting to www.appsloveworld.com (www.appsloveworld.com)|52.12.198.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [video/mp4]
Saving to: ‘640.mp4’

640.mp4                 [  <=>               ]   5.42M  24.7MB/s    in 0.2s    

2021-11-13 17:08:49 (24.7 MB/s) - ‘640.mp4’ saved [5683357]

2021/11/13 17:08:49 INFO  : Local file system at /content/drives/OneDrive/Downloads: Server side directory move failed - fallback to file moves: can't copy directory - destination already exists
2021/11/13 17:08:58 INFO  : 640.mp4: Copied (new)
2021/11/13 17:08:58 INFO  : 640.mp4: Deleted
2021/11/13 17:08:58 INFO  : 
Transferred:   	    5.420 MiB / 5.420 MiB, 100%, 792.761 KiB/s, ETA 0s
Checks:                 2 / 2, 100%
Deleted:                1 (files), 0 (dirs)
Ren